In [1]:
#多层感知机
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
#nn.Sequential定义了一种特殊的Module
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0888, -0.2694,  0.0615, -0.0786,  0.0928,  0.1440, -0.0636,  0.0950,
         -0.1150, -0.1867],
        [ 0.0802, -0.3250,  0.0373, -0.1510,  0.0858,  0.0995, -0.0585,  0.2217,
          0.0097, -0.2129]], grad_fn=<AddmmBackward>)

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
#实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层
net = MLP()
net(X)

tensor([[ 0.1153,  0.0441,  0.1708,  0.1995,  0.0695,  0.3596,  0.2647,  0.1446,
         -0.0404, -0.0334],
        [ 0.0748,  0.1681,  0.1336,  0.2394,  0.0388,  0.2274,  0.2329,  0.1074,
          0.0137,  0.1074]], grad_fn=<AddmmBackward>)

In [4]:
#顺序块
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1779,  0.0858, -0.1134, -0.3791, -0.0894,  0.0393, -0.0518, -0.1130,
          0.0816,  0.1126],
        [ 0.1377,  0.1960, -0.0566, -0.4475, -0.1611, -0.0721, -0.1214, -0.1755,
          0.1199,  0.0563]], grad_fn=<AddmmBackward>)

In [5]:
#在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.1715, grad_fn=<SumBackward0>)

In [6]:
#混合搭配各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0347, grad_fn=<SumBackward0>)